In [1]:
import sbibm
import torch
import math
import os
import ot
import pickle

import torch.distributions as D
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import wasserstein_distance_nd

/home/yppatel/anaconda3/envs/operator/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-20 09:16:03.171478: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-20 09:16:04.103945: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
task_name = "two_moons"
task = sbibm.get_task(task_name)  # See sbibm.get_available_tasks() for all tasks
prior = task.get_prior()
simulator = task.get_simulator()

In [3]:
cached_fn = os.path.join("trained", f"{task_name}.nf")
with open(cached_fn, "rb") as f:
    generative_model = pickle.load(f)
generative_model.to("cpu")

Flow(
  (_transform): CompositeTransform(
    (_transforms): ModuleList(
      (0): PiecewiseRationalQuadraticCouplingTransform(
        (transform_net): ResidualNet(
          (initial_layer): Linear(in_features=3, out_features=50, bias=True)
          (blocks): ModuleList(
            (0-1): 2 x ResidualBlock(
              (context_layer): Linear(in_features=2, out_features=50, bias=True)
              (linear_layers): ModuleList(
                (0-1): 2 x Linear(in_features=50, out_features=50, bias=True)
              )
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (final_layer): Linear(in_features=50, out_features=29, bias=True)
        )
      )
      (1): LULinear()
      (2): PiecewiseRationalQuadraticCouplingTransform(
        (transform_net): ResidualNet(
          (initial_layer): Linear(in_features=3, out_features=50, bias=True)
          (blocks): ModuleList(
            (0-1): 2 x ResidualBlock(
              (context_layer):

In [6]:
wass_dists = []
for obs_idx in range(1,11):
    observation = task.get_observation(num_observation=obs_idx)  # 10 per task
    ref_samples = task.get_reference_posterior_samples(num_observation=obs_idx).detach().cpu().numpy()
    K = len(ref_samples)
    var_samples = generative_model.sample(K, observation).detach().cpu().numpy()[0]

    a, b = np.ones((K,)) / K, np.ones((K,)) / K  # uniform distribution on samples
    M = ot.dist(ref_samples, var_samples)
    W = ot.emd2(a, b, M)

    wass_dists.append(W)

/home/yppatel/anaconda3/envs/operator/lib/python3.11/site-packages/nflows/transforms/lu.py:80: UserWarning: torch.triangular_solve is deprecated in favor of torch.linalg.solve_triangularand will be removed in a future PyTorch release.
torch.linalg.solve_triangular has its arguments reversed and does not return a copy of one of the inputs.
X = torch.triangular_solve(B, A).solution
should be replaced with
X = torch.linalg.solve_triangular(A, B). (Triggered internally at ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2192.)
  outputs, _ = torch.triangular_solve(
/home/yppatel/anaconda3/envs/operator/lib/python3.11/site-packages/ot/lp/__init__.py:571: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  check_result(result_code)
/home/yppatel/anaconda3/envs/operator/lib/python3.11/site-packages/ot/lp/__init__.py:571: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  check_result(result_code)
/home/yppatel/anaconda3/envs/operator/li

In [12]:
import cvxpy as cp

cp_idx = 8
alpha  = 0.2
q_hat  = np.quantile(wass_dists[:cp_idx], q=1-alpha)

In [13]:
p = np.random.randint(low=0, high=1000, size=(1, ref_samples.shape[-1]))[0]
u = np.random.uniform(low=0, high=1, size=1)[0]
B = np.random.uniform(np.max(p), np.sum(p) - u * np.max(p))

In [ ]:
subopt_gaps = []
for obs_idx in (cp_idx, 11):
    observation = task.get_observation(num_observation=obs_idx)  # 10 per task
    ref_samples = task.get_reference_posterior_samples(num_observation=obs_idx).detach().cpu().numpy()
    K = len(ref_samples)
    var_samples = generative_model.sample(K, observation).detach().cpu().numpy()[0]
    
    w    = cp.Variable(2)
    nominal_prob = cp.Problem(cp.Minimize(cp.mean(-ref_samples @ w)),
                    [w <= 1, w >= 0, p @ w <= B])
    nominal_opt = nominal_prob.solve()

    q_hat = 1.0
    w     = cp.Variable(2)
    var_prob = cp.Problem(cp.Minimize(cp.mean(-var_samples @ w) + q_hat * cp.norm(w, "inf")),
                    [w <= 1, w >= 0, p @ w <= B])
    var_opt = var_prob.solve()
    subopt_gaps.append((var_opt - nominal_opt) / nominal_opt)

In [8]:
var_opt

3.693161012146309e-10

In [9]:
nominal_opt

-0.8203821140439671